## create rasterized datasets

In [1]:
import sys

sys.path.append("..")
from utils.utils import PreprocessCylindricalAnnotations
import os

dataset = "jrc_22ak351-leaf-2l"
annotation_path = f"../annotations/{dataset}/"
annotation_csvs = [annotation_path + f for f in os.listdir(annotation_path)]

preprocess = PreprocessCylindricalAnnotations(
    annotation_csvs=annotation_csvs,
    dataset=dataset,
    validation_or_test_rois=[],
)
preprocess.extract_annotation_information()
preprocess.write_annotations_as_cylinders_and_get_intersections()
preprocess.write_intersection_mask()
# seems to work https://neuroglancer-demo.appspot.com/#!gs://flyem-user-links/short/2023-09-30.025524.json

  0%|          | 13/2906 [00:00<00:26, 111.23it/s]/groups/cellmap/cellmap/ackermand/Programming/plasmodesmata_dacapo/preprocessing/jrc_22_ak351-leaf-2l/../utils/utils.py:54: RuntimeWarning: invalid value encountered in divide
  d = np.divide(end_2 - end_1, np.linalg.norm(end_2 - end_1))
  1%|▏         | 39/2906 [00:03<04:19, 11.04it/s]/groups/cellmap/cellmap/ackermand/Programming/plasmodesmata_dacapo/preprocessing/jrc_22_ak351-leaf-2l/../utils/utils.py:187: UserWarning: Empty plasmodesmata #40 ([6949.04589844 4345.22607422 2007.22814941]-[6949.04589844 4345.22607422 2007.22814941]) will be ignored
  warnings.warn(
100%|██████████| 2906/2906 [04:42<00:00, 10.30it/s]


In [2]:
preprocess.remove_validation_or_test_annotations_from_training()

100%|██████████| 4427/4427 [00:00<00:00, 10779.14it/s]

number of original centers: 4427, number of training centers: 3438


In [9]:
preprocess.write_out_removed_annotations()
# preprocess.visualize_removed_annotations(original_validation_bounding_box)

100%|██████████| 989/989 [00:00<00:00, 158994.51it/s]

precomputed://https://cellmap-vm1.int.janelia.org/dm11/ackermand/Programming/plasmodesmata_dacapo/preprocessing/removed_annotations


# Dacapo

In [4]:
from dacapo.experiments.architectures import CNNectomeUNetConfig
from dacapo.experiments.trainers import GunpowderTrainerConfig
from dacapo.experiments.trainers.gp_augments import (
    ElasticAugmentConfig,
    IntensityAugmentConfig,
)
from dacapo.experiments.tasks import AffinitiesTaskConfig
from funlib.geometry.coordinate import Coordinate
import math

## Trainer

In [5]:
trainer_config = GunpowderTrainerConfig(
    name="default_v2_no_dataset_predictor_node_lr_5E-5",
    batch_size=2,
    learning_rate=0.00005,
    augments=[
        ElasticAugmentConfig(
            control_point_spacing=(100, 100, 100),
            control_point_displacement_sigma=(10.0, 10.0, 10.0),
            rotation_interval=(0, math.pi / 2.0),
            subsample=8,
            uniform_3d_rotation=True,
        ),
        IntensityAugmentConfig(
            scale=(0.7, 1.3),
            shift=(-0.2, 0.2),
            clip=True,
        ),
    ],
    clip_raw=True,
    num_data_fetchers=20,
    snapshot_interval=10000,
    min_masked=0.05,
    add_predictor_nodes_to_dataset=False,
)

## Task

In [6]:
task_config = AffinitiesTaskConfig(
    name=f"3d_lsdaffs_weight_ratio_1.00",
    neighborhood=[
        (1, 0, 0),
        (0, 1, 0),
        (0, 0, 1),
        (3, 0, 0),
        (0, 3, 0),
        (0, 0, 3),
        (9, 0, 0),
        (0, 9, 0),
        (0, 0, 9),
    ],
    lsds=True,
    lsds_to_affs_weight_ratio=1.00,
)

## Architecture

I had an issue where, by default, I created the rasterization at the same resolution as the raw data. But the default architecture (with the upsampling layer `upsample_factors`) expects it to be at 2x the resolution including mask and validation. This resulted in an error when submitting. Since we don't really care about a higher res (at the moment), we can just comment out the upsampling layer (`constant_upsample` and `upsample_factors`)

In [7]:
architecture_config = CNNectomeUNetConfig(
    name="unet",
    input_shape=Coordinate(216, 216, 216),
    eval_shape_increase=Coordinate(72, 72, 72),
    fmaps_in=1,
    num_fmaps=12,
    fmaps_out=72,
    fmap_inc_factor=6,
    downsample_factors=[(2, 2, 2), (3, 3, 3), (3, 3, 3)],
    # constant_upsample=True,
    # upsample_factors=[(2, 2, 2)],
)

## Datasplit

EVERYTHING MUST BE IN Z,Y,X AND NM!

In [12]:
# # use centers
# from pathlib import Path
# from dacapo.experiments.datasplits.datasets.arrays import (
#     ZarrArrayConfig,
#     IntensitiesArrayConfig,
#     CropArrayConfig,
# )
# from dacapo.experiments.datasplits.datasets import RawGTDatasetConfig
# from dacapo.experiments.datasplits import TrainValidateDataSplitConfig
# from funlib.geometry import Roi

# raw_config = ZarrArrayConfig(
#     name="raw",
#     file_name=Path("/nrs/stern/em_data/jrc_22ak351-leaf-3m/jrc_22ak351-leaf-3m.n5"),
#     dataset="em/fibsem-uint8/s0",
# )
# # We get an error without this, and will suggests having it as such https://cell-map.slack.com/archives/D02KBQ990ER/p1683762491204909
# raw_config = IntensitiesArrayConfig(
#     name="raw", source_array_config=raw_config, min=0, max=255
# )

# gt_config = ZarrArrayConfig(
#     name="plasmodesmata",
#     file_name=Path("/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5"),
#     dataset="plasmodesmata_as_cylinders",
# )

# # mask out regions of overlapping plasmodesmata
# mask_config = ZarrArrayConfig(
#     name="mask",
#     file_name=Path("/nrs/cellmap/ackermand/cellmap/leaf-gall/masks.zarr"),
#     dataset="jrc_22ak351-leaf-3m",
# )

# # could do validation as a file
# # val_gt_config = ZarrArrayConfig(
# #     name="plasmodesmata", file_name="/path/to/data.zarr", dataset="labels_val"
# # )

# # NOTE: Everything has to be in z,y,x
# validation_roi = Roi(offset[::-1] * resolution, dimensions[::-1] * resolution)
# val_gt_config = CropArrayConfig(
#     "val_gt", source_array_config=gt_config, roi=validation_roi
# )
# training_data_config = RawGTDatasetConfig(
#     "train",
#     raw_config=raw_config,
#     gt_config=gt_config,
#     sample_points=[
#         Coordinate(pd_center[::-1]) for pd_center in updated_pd_centers
#     ],  # [Coordinate((29229*8,1862*8,7439*8))], #
#     mask_config=mask_config,
# )
# validation_data_config = RawGTDatasetConfig(
#     "val", raw_config=raw_config, gt_config=val_gt_config, mask_config=mask_config
# )
# datasplit_config = TrainValidateDataSplitConfig(
#     name="plasmodesmata",
#     train_configs=[training_data_config],
#     validate_configs=[validation_data_config],
# )

In [8]:
# use pseudorandom centers
from pathlib import Path
from dacapo.experiments.datasplits.datasets.arrays import (
    ZarrArrayConfig,
    IntensitiesArrayConfig,
    CropArrayConfig,
)
from dacapo.experiments.datasplits.datasets import RawGTDatasetConfig
from dacapo.experiments.datasplits import TrainValidateDataSplitConfig
from funlib.geometry import Roi

raw_config = ZarrArrayConfig(
    name="raw",
    file_name=Path("/nrs/stern/em_data/jrc_22ak351-leaf-3m/jrc_22ak351-leaf-3m.n5"),
    dataset="em/fibsem-uint8/s0",
)
# We get an error without this, and will suggests having it as such https://cell-map.slack.com/archives/D02KBQ990ER/p1683762491204909
raw_config = IntensitiesArrayConfig(
    name="raw", source_array_config=raw_config, min=0, max=255
)

gt_config = ZarrArrayConfig(
    name="plasmodesmata",
    file_name=Path("/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5"),
    dataset="plasmodesmata_as_cylinders",
)

# mask out regions of overlapping plasmodesmata
mask_config = ZarrArrayConfig(
    name="mask",
    file_name=Path("/nrs/cellmap/ackermand/cellmap/leaf-gall/masks.zarr"),
    dataset="jrc_22ak351-leaf-3m",
)

# could do validation as a file
# val_gt_config = ZarrArrayConfig(
#     name="plasmodesmata", file_name="/path/to/data.zarr", dataset="labels_val"
# )

# NOTE: Everything has to be in z,y,x
validation_roi = Roi(
    original_validation_bounding_box.offset[::-1] * preprocess.resolution,
    original_validation_bounding_box.shape[::-1] * preprocess.resolution,
)
val_gt_config = CropArrayConfig(
    "val_gt", source_array_config=gt_config, roi=validation_roi
)
training_data_config = RawGTDatasetConfig(
    f"train",
    raw_config=raw_config,
    gt_config=gt_config,
    sample_points=[
        Coordinate(preprocess.pseudorandom_training_center[::-1])
        for preprocess.pseudorandom_training_center in preprocess.pseudorandom_training_centers
    ],  # [Coordinate((29229*8,1862*8,7439*8))], #
    mask_config=mask_config,
)
validation_data_config = RawGTDatasetConfig(
    "val", raw_config=raw_config, gt_config=val_gt_config, mask_config=mask_config
)
datasplit_config = TrainValidateDataSplitConfig(
    name=f"plasmodesmata_pseudorandom_training_centers_maxshift_18_more_annotations",
    train_configs=[training_data_config],
    validate_configs=[validation_data_config],
)

## Run

In [10]:
from dacapo.experiments import RunConfig
from dacapo.experiments.starts import StartConfig
from dacapo.store.create_store import create_config_store

config_store = create_config_store()

start_config = StartConfig("setup04", "best")
iterations = 200000
validation_interval = 5000
repetitions = 3
for i in range(repetitions):
    run_config = RunConfig(
        name=("_").join(
            [
                "scratch" if start_config is None else "finetuned",
                task_config.name,
                datasplit_config.name,
                architecture_config.name,
                trainer_config.name,
            ]
        )
        + f"__{i}",
        task_config=task_config,
        datasplit_config=datasplit_config,
        architecture_config=architecture_config,
        trainer_config=trainer_config,
        num_iterations=iterations,
        validation_interval=validation_interval,
        repetition=i,
        start_config=start_config,
    )
    config_store.store_run_config(run_config)
# "dacapo run -r {run_config.name}"

In [11]:
print(run_config.name)

finetuned_3d_lsdaffs_weight_ratio_1.00_plasmodesmata_pseudorandom_training_centers_maxshift_18_more_annotations_unet_default_v2_no_dataset_predictor_node_lr_5E-5__2


# Prediction Mask

In [7]:
from funlib.persistence import open_ds, prepare_ds
from funlib.geometry import Roi, Coordinate
from scipy.ndimage import binary_dilation, distance_transform_edt
import numpy as np

for iterations in range(1, 4):
    ds = open_ds(
        f"/nrs/cellmap/ackermand/cellmap/leaf-gall/{dataset}.n5",
        "plasmodesmata_column_cells",
    )
    voxel_size = ds.voxel_size
    data = ds.to_ndarray() > 0
    ds = open_ds(
        f"/nrs/cellmap/ackermand/cellmap/leaf-gall/{dataset}.n5",
        "plasmodesmata_column_target_cells",
    )
    data += ds.to_ndarray() > 0
    data = 1 - (data > 0)
    data = binary_dilation(data, iterations=iterations)

    output_ds = prepare_ds(
        "/nrs/cellmap/ackermand/cellmap/leaf-gall/prediction_masks.zarr",
        f"dilation_iterations_{iterations}_{dataset}",
        total_roi=ds.roi,
        voxel_size=voxel_size,
        dtype=np.uint8,
        write_size=Coordinate(np.array([64, 64, 64]) * 256),
        delete=True,
        # force_exact_write_size=True
    )
    output_ds[ds.roi] = data